In [89]:
import numpy as np
import pandas as pd 
import urllib
import re
import helper
import random

In [97]:
# Run this to reload helper.py so you don't have to restart the kernel
import importlib

importlib.reload(helper)

<module 'helper' from '/Users/prakash/Desktop/Courses/ML/loan-sharks/poems/helper.py'>

## Preprocess Data

In [91]:
all_words, all_sequences, word_dict, all_sonnet_int, syllable_dict= helper.getAllWordsAndSequences("data/shakespeare.txt", "data/Syllable_dictionary.txt",syllable_count=True)

In [92]:
all_sonnet_int[11]

[905, 2956, 96, 480, 2870, 2070, 2474, 3205]

## Utility function

In [93]:
def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r

## Naive HMM

In [94]:
all_sonnet_int_new=[]
for i in range(0,len(all_sonnet_int),14):
    temp=[]
    for j in range(14):
        temp.extend(all_sonnet_int[i+j])
    all_sonnet_int_new.append(temp)

In [95]:
hmm_naive = helper.unsupervised_HMM(all_sonnet_int_new, 10, 500)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100


In [98]:
print('Sample Sentence:\n====================')
print(helper.sample_sentence(hmm_naive, word_dict, n_words=200))

Sample Sentence:
Full friend 
 make aught perfect that and now ruining 
 bear still 
 that flattery and costly gav'st moan on which what dignified 
 say thou of i forbear makes hast doom of blushing 
 call proof 
 to heaven honouring her the thee jacks but all of truth 
 her accents 
 or praises whose mine that from no they 
 cannot three my my 
 thee a pleasure 
 but so 
 thee thou of self-substantial jealousy may like less 
 thou him hours to which but verse flattered and o'er-green my universe be night 
 but like love be sober should what blessed of set familiar her what as are from i 
 fawn 
 for 
 as adjunct 
 why hawks appear 
 be wide love husbandry 
 in miracle tongue-tied me line proud 
 be 
 to else is is the with what i plague from mine a 
 as kind new thy fortune's sight 
 peep eyes although my 
 the i it be the seals want tempests thou wilt they in or a alone 
 the days it morrow the in some wretched cannot pity how his smell 
 when decrepit they...


## Training HMM in reverse

In [99]:
new_sonet_sequence=[]
for idx in range(len(all_sonnet_int)):
    if len(all_sonnet_int[idx])>1:
        new_sonet_sequence.append(all_sonnet_int[idx][-2::-1])

In [100]:
hmm_reverse = helper.unsupervised_HMM(new_sonet_sequence, 30, 100)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100


## Rhyme

In [101]:
rhyme=[]
rhyme_couplet=[]
new_sonet_sequence=[]
for idx in range(len(all_sonnet_int)):
    if (idx+2)%14==0 and len(all_sonnet_int[idx+1])>1:
        rhyme_couplet.append((all_sonnet_int[idx][-2],all_sonnet_int[idx+1][-2]))
    elif idx%4==0 and len(all_sonnet_int[idx+2])>1:
        rhyme.append((all_sonnet_int[idx][-2],all_sonnet_int[idx+2][-2]))
        rhyme.append((all_sonnet_int[idx+1][-2],all_sonnet_int[idx+3][-2]))

In [102]:
def sample_rev_sonents(hmm, obs_map, syllable_dict, n_sonnet=10, start_word=None):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)
    start_state = None
    if start_word:
        emition_pos=[hmm.O[i][start_word] for i in range(hmm.L)]
        start_state=random.choices([i for i in range(hmm.L)],weights=emition_pos)[0]
    # Sample and convert sentence.
    emission = []
    states = []
    count_sonet = int(syllable_dict[obs_map_r[start_word]][-1])
    state=start_state
#     print(obs_map_r[start_word])

    for i in range(100):
            # Append state.
            states.append(state)
            while(True):

                # Sample next observation.
                rand_var = random.uniform(0, 1)
                next_obs = 0

                while rand_var > 0:
                    rand_var -= hmm.O[state][next_obs]
                    next_obs += 1

                next_obs -= 1
                emission.append(next_obs)
                if i!=0:
                    count_sonet+=int(syllable_dict[obs_map_r[next_obs]][-1])
                if count_sonet==10:
                    break
                elif count_sonet>10:
                    count_sonet-=int(syllable_dict[obs_map_r[next_obs]][-1])
                    emission.pop()
                    continue
                else:
                    break
            if count_sonet==10:
                break

            # Sample next state.
            rand_var = random.uniform(0, 1)
            next_state = 0

            while rand_var > 0:
                rand_var -= hmm.A[state][next_state]
                next_state += 1

            next_state -= 1
            state = next_state
    sentence = [obs_map_r[i] for i in emission]
    
    if start_word:
        sentence[0]=obs_map_r[start_word]

    return ' '.join(sentence[::-1]).capitalize()

In [103]:
print('Sample Sonnet:\n====================')
sentence_rhyme=['' for i in range(14)]
mapper=obs_map_reverser(word_dict)
for i in range(3):
    new_rhyme_1=random.choice(rhyme)
    new_rhyme_2=random.choice(rhyme)
    sentence_rhyme[i*4]=new_rhyme_1[0]
    sentence_rhyme[i*4+1]=new_rhyme_2[0]
    sentence_rhyme[i*4+2]=new_rhyme_1[1]
    sentence_rhyme[i*4+3]=new_rhyme_2[1]
#     print(new_rhyme_1,new_rhyme_2)
new_rhyme_1=random.choice(rhyme_couplet)
sentence_rhyme[-2]=new_rhyme_1[0]
sentence_rhyme[-1]=new_rhyme_1[1]
# print(mapper[new_rhyme_1[0]],mapper[new_rhyme_1[1]])
for start_rhyme in sentence_rhyme:
    print(sample_rev_sonents(hmm_reverse, word_dict, syllable_dict, n_sonnet=10, start_word=start_rhyme))

Sample Sonnet:
Ne'er love leave be hand in another change
Soul my roses for you taste fair course gone
Self to fly o me age his damasked strange
Self nor for memorial numbers alone
Eyes like other discased than would disarmed
Was i than you she truth posterity
Thy monument when thy perpetual
As to thou doth dry thou ride so the heir
I of thy impression of because knife
Arising other verse so to whence ill
On made beseechers list goodness when life
Pleasure shall lusty love confined again
Deem over-partial two and blame your life
To but me scope nor perjured and thy knife


## Meter

In [104]:
import nltk
nltk.download('cmudict')

[nltk_data] Downloading package cmudict to /Users/prakash/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [105]:
from nltk.corpus import cmudict

In [106]:
def stress_syllable(word,prondict):
    start,end=-1,-1
    if word not in prondict:
        return start,end
    processed=[ele[-1] for ele in prondict[word][0]]
    for ele in processed:
        if ele.isnumeric():
            start=int(ele)
            break
    for ele in processed[::-1]:
        if ele.isnumeric():
            end=int(ele)
            break
    return start,end

In [107]:
prondict = cmudict.dict()
stress_syllable('compare',prondict)

(0, 1)

In [108]:
def sample_rev_sonents_meter(hmm, obs_map, syllable_dict, prondict, n_sonnet=10, start_word=None):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)
    start_state = None
    if start_word:
        emition_pos=[hmm.O[i][start_word] for i in range(hmm.L)]
        start_state=random.choices([i for i in range(hmm.L)],weights=emition_pos)[0]
    # Sample and convert sentence.
    emission = []
    states = []
    count_sonet = int(syllable_dict[obs_map_r[start_word]][-1])
    start,_ = stress_syllable(obs_map_r[start_word],prondict)
    state=start_state
#     print(obs_map_r[start_word])

    for i in range(100):
            # Append state.
            states.append(state)
            while(True):

                # Sample next observation.
                rand_var = random.uniform(0, 1)
                next_obs = 0

                while rand_var > 0:
                    rand_var -= hmm.O[state][next_obs]
                    next_obs += 1

                next_obs -= 1
                start_new,end_new = stress_syllable(obs_map_r[next_obs],prondict)
                if end_new!=start:
                    continue
                start=start_new
                emission.append(next_obs)
                if i!=0:
                    count_sonet+=int(syllable_dict[obs_map_r[next_obs]][-1])
                if count_sonet==10:
                    break
                elif count_sonet>10:
                    count_sonet-=int(syllable_dict[obs_map_r[next_obs]][-1])
                    emission.pop()
                    continue
                else:
                    break
            if count_sonet==10:
                break

            # Sample next state.
            rand_var = random.uniform(0, 1)
            next_state = 0

            while rand_var > 0:
                rand_var -= hmm.A[state][next_state]
                next_state += 1

            next_state -= 1
            state = next_state
    sentence = [obs_map_r[i] for i in emission]
    
    if start_word:
        sentence[0]=obs_map_r[start_word]

    return ' '.join(sentence[::-1]).capitalize()

In [109]:
print('Sample Sonnet:\n====================')
sentence_rhyme=['' for i in range(14)]
mapper=obs_map_reverser(word_dict)
for i in range(3):
    new_rhyme_1=random.choice(rhyme)
    new_rhyme_2=random.choice(rhyme)
    sentence_rhyme[i*4]=new_rhyme_1[0]
    sentence_rhyme[i*4+1]=new_rhyme_2[0]
    sentence_rhyme[i*4+2]=new_rhyme_1[1]
    sentence_rhyme[i*4+3]=new_rhyme_2[1]
#     print(new_rhyme_1,new_rhyme_2)
new_rhyme_1=random.choice(rhyme_couplet)
sentence_rhyme[-2]=new_rhyme_1[0]
sentence_rhyme[-1]=new_rhyme_1[1]
# print(mapper[new_rhyme_1[0]],mapper[new_rhyme_1[1]])
for start_rhyme in sentence_rhyme:
    print(sample_rev_sonents_meter(hmm_reverse, word_dict, syllable_dict, prondict, n_sonnet=10, start_word=start_rhyme))

Sample Sonnet:
Be that can come is of with burn his aid
Keeps i look wrought for never despair gay
On live by from is am voices decayed
Check mad that tongues i through not world or spend
All chide thou use you we bad must subdued
Preserve my i by be are friends true shine
Make to then than my sweet i thy renewed
My love like i for make mine time that mine
So dignity the because little come
From same winds but put have how my not hour
I so twice love all love with growing sum
Things made world part with when verse nor his sour
Art said she though this my subjects and be
With dear such their gilded returned ten thee


## Mixing Spenser

In [110]:
all_words, all_sequences, word_dict, all_sonnet_int, syllable_dict= helper.getAllWordsAndSequences("data/shakespeare.txt", "data/Syllable_dictionary.txt",syllable_count=True)

In [111]:
all_words, all_sequences, word_dict, all_sonnet_int, syllable_dict= helper.addSpenserData("data/spenser.txt",all_words, all_sequences, word_dict, all_sonnet_int, syllable_dict)

In [112]:
new_sonet_sequence=[]
for idx in range(len(all_sonnet_int)):
    if len(all_sonnet_int[idx])>1:
        new_sonet_sequence.append(all_sonnet_int[idx][-2::-1])

In [114]:
hmm_spenser = helper.unsupervised_HMM(new_sonet_sequence, 10, 100)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100


In [115]:
rhyme=[]
rhyme_couplet=[]
new_sonet_sequence=[]
for idx in range(len(all_sonnet_int)):
    if (idx+2)%14==0 and len(all_sonnet_int[idx+1])>1:
        rhyme_couplet.append((all_sonnet_int[idx][-2],all_sonnet_int[idx+1][-2]))
    elif idx%4==0 and len(all_sonnet_int[idx+2])>1 and len(all_sonnet_int[idx+1])>1:
        rhyme.append((all_sonnet_int[idx][-2],all_sonnet_int[idx+2][-2]))
        rhyme.append((all_sonnet_int[idx+1][-2],all_sonnet_int[idx+3][-2]))

In [116]:
prondict = cmudict.dict()
stress_syllable('compare',prondict)

(0, 1)

In [117]:
print('Sample Sonnet:\n====================')
sentence_rhyme=['' for i in range(14)]
mapper=obs_map_reverser(word_dict)
for i in range(3):
    new_rhyme_1=random.choice(rhyme)
    new_rhyme_2=random.choice(rhyme)
    sentence_rhyme[i*4]=new_rhyme_1[0]
    sentence_rhyme[i*4+1]=new_rhyme_2[0]
    sentence_rhyme[i*4+2]=new_rhyme_1[1]
    sentence_rhyme[i*4+3]=new_rhyme_2[1]
#     print(new_rhyme_1,new_rhyme_2)
new_rhyme_1=random.choice(rhyme_couplet)
sentence_rhyme[-2]=new_rhyme_1[0]
sentence_rhyme[-1]=new_rhyme_1[1]
# print(mapper[new_rhyme_1[0]],mapper[new_rhyme_1[1]])
for start_rhyme in sentence_rhyme:
#     print(mapper[start_rhyme])
    print(sample_rev_sonents_meter(hmm_spenser, word_dict, syllable_dict, prondict, n_sonnet=10, start_word=start_rhyme))

Sample Sonnet:
Be old will is love have dost desire
Scope i have proud comments dismay their brood
Buy self they with my it life she not heat
Breast when i my if as lends for mine blood
Most i that so that till those chastity
To which once me hell not brain of well live
You fairly her within one's monument
Flame care of as chaste look i deep but give
Foe that thus thy do not miss knowing slow
Your my which vain let make to bends through leave
Forth strong thee castles untainted expense
His my woman and another deceive
Their as seem prove ill for my fit to prove
Be age of love which though can your those love
